# PV Degradation Tools Tutorial - 1
### Module Humidity and Edge Seal Width
***
Matthew Brown

2023.06.02
***

**Requirements:**
- weather file (psm3 preferred) demo file is provided

**Objectives:**
1. Read in necessary weather data
2. Generate solar position, POA, and module temperature
3. Generate module humidities
4. Calculate edge seal width

In [1]:
import os
import pandas as pd

import pvdeg 
from pvdeg import TEST_DATA_DIR

## 1.
From a psm3 (or TMY3 or EPW) file we need the following:
- DNI
- DHI
- GHI
- Temperature, Dry-Bulb
- Temperature, Dew-Point
- Wind Speed
- Relative Humidity
- Latitude
- Longitude
- altitude

In [2]:
PSM_FILE = os.path.join(TEST_DATA_DIR,'psm3_pytest.csv')
WEATHER, META = pvdeg.weather.read(PSM_FILE,'psm')

dni = WEATHER.dni
dhi = WEATHER.dhi
ghi = WEATHER.ghi

time_range = WEATHER.index

## 2. Solar Position, POA, and Module Temperature

Many operations with `PVDEG` require solar position, POA, and module temperature. While there is usually a method that calculates these data sets automatically, they can be calculated directly. Here, we generate the solar position (azimuth, elevation, etc), plane of array irradiance (POA), and module temperature. `PVDEG` has wrappers for quickly using `PVLIB` to generate these figures with minimal input

In [3]:
sol_pos = pvdeg.spectral.solar_position(weather_df=WEATHER, meta=META)

poa_df = pvdeg.spectral.poa_irradiance(weather_df=WEATHER, meta=META, solar_position=sol_pos)

temp_mod = pvdeg.temperature.module(weather_df=WEATHER, poa=poa_df)

Module temperature has several optional parameters. Below is an example which explicity uses all of the default values. For more options, see the pvlib documentation for temperature models.

https://pvlib-python.readthedocs.io/en/stable/reference/pv_modeling/temperature.html

In [7]:
temp_mod = pvdeg.temperature.module(weather_df = WEATHER, poa = poa_df,
                                    temp_model = 'sapm',
                                    conf= 'open_rack_glass_polymer',
                                    wind_speed_factor = 1)

## 3. Module Humidities
`PVDEG` can be used to calculate the relative humidity of several layers within a PV module. This can be done with 2 techniques.

a. Automatic
    
b. Manual
    

### 3.a. Automatic
Use the function `calc_rel_humidity`. This method does all of the calculations behind the scenes. It requires only 2 inputs (weather data and meta data)

In [6]:
rh_module = pvdeg.humidity.calc_rel_humidity(weather_df=WEATHER, meta=META)
rh_module.head()

,RH_surface_outside,RH_front_encap,RH_back_encap,RH_backsheet
1999-01-01 00:30:00-07:00,79.39,44.198787,79.390000,79.390000
1999-01-01 01:30:00-07:00,80.84,44.198787,79.415220,80.127610
1999-01-01 02:30:00-07:00,82.98,44.198787,79.477223,81.228611
1999-01-01 03:30:00-07:00,85.01,44.198787,79.573455,82.291727
1999-01-01 04:30:00-07:00,85.81,44.198787,79.681927,82.745964


### 3.b. Manual
Use the individual functions to calculate the RH in each layer. These are named and require more extensive input parameters. This must be done in the correct order:
1. front encapsulant
2. back encapsulant
3. backsheet

In [10]:
rh_front_encap = pvdeg.humidity.rh_front_encap(rh_ambient=WEATHER['relative_humidity'],
                                               temp_ambient=WEATHER['temp_air'],
                                               temp_module=temp_mod)

rh_back_encap = pvdeg.humidity.rh_back_encap(rh_ambient=WEATHER['relative_humidity'],
                                             temp_ambient=WEATHER['temp_air'],
                                             temp_module=temp_mod)

rh_backsheet = pvdeg.humidity.rh_backsheet_from_encap(rh_back_encap=rh_back_encap,
                                                      rh_surface_outside=WEATHER['relative_humidity'])

## 4. 
### Design: Edge Seal Width
`pvdeg.design.edge_seal_width` calculates the width [cm] required for several years of water ingress. If you do not specify the number of years, it will use the default value of 25.

In [4]:
psat, psat_avg = pvdeg.humidity.psat(WEATHER['temp_air'])

k = pvdeg.design.k(avg_psat=psat_avg)

edge_seal_width = pvdeg.design.edge_seal_width(k, years=20)

print(psat_avg)
print(k)
print(edge_seal_width)

1.3946499092628233
0.00153182106297676
0.6413923384461668


### Alternate Edge Seal Width
If you have the dew point, or wet-bulb temperature, you can calculate the edge seal width directly.

In [6]:
edge_seal_width = pvdeg.design.edge_seal_from_dew_pt(dew_pt_temp=WEATHER['Dew Point'], years=20)
edge_seal_width

0.40245072298649226